__Вопросы для повторения__:

* Что такое виртуальная функция?
* Что здесь будет напечатано и почему? Как работает механизм вызова виртуальных функций?

```c++
struct Base
{
    virtual void f() { std::cout << "Base::f" << std::endl; }
    void g()         { std::cout << "Base::g" << std::endl; }
    Base() {
        std::cout << "Base" << std::endl;
        f();
    }
    ~Base() { std::cout << "~Base" << std::endl }
};

struct D: Base
{
    D() { std::cout << "D" << std::endl; }
    ~D() { std::cout << "~D" << std::endl; }
    void f() override { std::cout << "D::f" << std::endl; }
    void g() override { std::cout << "D::g" << std::endl; }
};

void func_1()
{
    Base *p = new D;
    p->f(); p->g();
    delete p;

    Base b = D();
    b.f();
    b.g();
}
```

<details>
<summary>Подсказка</summary>
<p>

Во-первых, не скомпилируется из-за `void g() override`. Что такое override?
Во-вторых - см. обычные правила. Спрашивать за каждый пункт, почти везде грабля.

</p>
</details>

* Чему равен `sizeof(D)`? А поподробнее, зачем это нужно?

* Что такое `const`-методы?
* Что такое `mutable`? Когда он оправдан?
* Что напечатается? Какие правила вызова?

```c++
class C
{
public:
    void f()       { std::cout << "f()"       << std::endl; }
    void f() const { std::cout << "f() const" << std::endl; }
};

void func()
{
    C a1;
    const C a2;
    const C& a3 = a1;
    a1.f();
    a2.f();
    a3.f();
}
```

* Каков layout класса? (+ size, alignment)

```c++
struct S
{
    float x;
    std::uint64_t y;
    float z;
};
```
* Как починить?
* А в таком варианте?

```c++
struct S
{
    float x;
    std::uint64_t y;
    float z;
    
    virtual void g();
};
```

* Чему равен sizeof такой структуры?

```c++
struct S {};
```

* А такой?

```c++
struct S {};

struct D : S {};
```

https://en.cppreference.com/w/cpp/language/ebo


* class vs struct, примеры
* про что Single Responsibility Principle?
* про что Liskov Substitution Principle?

<br />

##### static-поля класса

Класс может иметь `static`-поля и `static`-методы:

`static`-поля можно рассматривать как глобальные константы / переменные:

```c++
class Animal
{
public:
    Animal(const std::string& a_name, unsigned a_age);
    ...
    
    // declaration
    static const std::string name_of_unknown;    
};

// definition
static const std::string Animal::name_of_unknown = "UNK";

// usage:
void func()
{
    std::cout << "name of uknown animal is: " << Animal::name_of_unknown << std::endl;
    
    Animal animal(Animal::name_of_unknown, 12);
}
```

На `static`-поля рпспространяются модификаторы области доступа `private`/`protected`/`public`:
* В случае `public` `static`-поле ведёт себя как обычная глобальная константа
* В случае `private` только методы класса имеют доступ до этой константы

__Вопрос__: 

<details>
<summary>Зачем могут быть полезны `static`-поля класса</summary>
<p>
   
* Классоспецифичные константы.
* Классоспецифичные глобальные переменные (по возможности избегать).
   
</p>
</details>


<details>
<summary>Ещё примеры?</summary>
<p>
   
* Класс DateTime. Можно в `public` вынести константы minute, second, hour ...
* Ещё?
   
</p>
</details>

<br />

##### static-методы класса

`static`-метод класса можно рассматривать как свободную friend-функцию:

```c++
class Animal
{
    int age = 0;

public:
    static void grow_up(int count, Animal& an)
    {
        an.age += count;
    }

    int get_age() const { return age; }
};

int main()
{
    Animal an;
    Animal::grow_up(4, an);  // TODO: обратить внимание на вызов
    std::cout << an.get_age() << std::endl;
    return 0;
}
```

`static`-метод - обычная friend-функция, находящаяся в "пространстве имён" класса.

**Для вызова static-метода _НЕ_ нужен объект класса**

Пример использования `static`-метода:

```c++
class House
{
    Floor floor;
    Roof roof;
    std::vector<Wall> walls;

public:
    House(Floor f, Roof r, std::vector<Wall> walls);
    
    static House make(const Config& config)
    {
        auto floor = Floor::make(config["floor"]);
        auto root = Roof::make(config["roof"]);
        auto walls = ...;        
        return House(floor, roof, walls);
    }    
};
```

__Вопрос__: что-нибудь напоминает?

__Вопрос__: когда оправдано использовать static-методы?

<br />

### Лекция 5. Шаблоны

Выдать домашнее задание по полиному

<br />

##### Какая идея стоит за шаблонами

Ранее мы познакомились с возможностью перегрузки функций. Давайте вспомним её на примере swap:

```c++
// поменять местами два int
void my_swap(int& a, int& b)
{
    int tmp = a;
    a = b;
    b = tmp;    
}

// поменять местами два short
void my_swap(short& a, short& b)
{
    short tmp = a;
    a = b;
    b = tmp;
}

// поменять местами два float
void my_swap(float& a, float& b)
{
    float tmp = a;
    a = b;
    b = tmp;
}

...
```

Вечер начинает быть томным ...

Для решения проблем написания одинакового кода придуманы шаблоны:

```c++
// напишем шаблон - как должна выглядеть функция
template<typename T>
void my_swap(T& a, T& b)
{
    Type tmp = a;
    a = b;
    b = tmp;
}
```

Применение шаблона:

```c++
int a = 3, b = 5;

// вызов my_swap(int&, int&), тип T указывается программистом явно
my_swap<int>(a, b);

// вызов my_swap(int&, int&), тип T выводится компилятором автоматически
my_swap(a, b); 


float x = 3.f, y = 5.f;
my_swap(x, y);
my_swap<float>(x, y);
```

Важное свойство шаблонов по сравнению с перегрузкой функций: шаблон компилируется только тогда, когда он вызывается. И компилируется только для тех типов, для которых он вызывается.

_но в каждом cpp-файле шаблон компилируется снова и снова_

Показать пример на godbolt.org, позакомментировать функции, продемонстрировать разницу в выхлопе компилятора.

```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Особенности шаблонов по сравнению с перегруженными функциями:
* компилируется только то, что инстанциируется в коде
* компилируется столько раз, в скольки единицах трансляции инстанциируется:
    * можно в одном cpp-файле 10 раз позвать myswap(int&, int&) - эта функция скомпилируется единожды
    * можно в 10 cpp-файлах один раз позвать myswap(int&, int&) - эта функция скомпилируется 10 раз
* накладные расходы во время компиляции на кодогенерацию при истанциации
* позволяет компилятору агрессивнее оптимизировать. Раскомментировать `__attribute__((noinline))` из примера и показать какой код сгенерирует компилятор. Объяснить, почему.
* позволяет нарушать ODR

Коротко:

* (+) меньше кода
* (+) быстрее
* (-) дольше компилируется
* (-) сложнее писать

__Вопросы__:
* Где поместить шаблонную функцию, которую нужно использовать в разных cpp-файлах?
* Где поместить её реализацию?
* Может ли шаблонная функция содержать некомпилирующийся код?

<br />

##### Специализация

Перегрузка функций позволяла сделать `myswap` у `std::string` более эффективно, без лишнего копирования памяти:

```c++
void myswap(int& a, int& b) { ... }
void myswap(short& a, short& b) { ... }

void myswap(std::string& a, std::string& b)
{
    a.swap(b);
}
```

Шаблоны тоже позволяют специализировать поведение функций, если наложить на шаблонный параметр ограничение, например:

(закинуть этот код на godbolt, показать во что компилируется программа)

```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

template<>
void __attribute__ ((noinline)) myswap<std::string>(std::string& a, std::string& b)
{
    a.swap(b);
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Во-первых, шаблон может иметь несколько параметров, а во-вторых, параметры не обязаны быть типами. Они могут быть, например, целыми числами:

```c++
template<int N, typename T>
T add_value(T x)
{
    return x + N;
}

int a = add_value<5>(100);

// 1. шаблон специфицирован программистом частично, тип Т компилятор определит сам
// 2. параметром шаблона выступает целое число.
```

__Вопрос__: Какую информацию здесь компилятор использует, чтобы вывести тип `T`?

<br />

##### Шаблонные классы

Аналогично функциям, классы тоже могут быть шаблонными:

Пример структуры:

```c++
// N-мерный вектор из курса линейной алгебры типа T
template<typename T, int N>
struct VectorN
{
    T data[N];
};
// TODO: обсудить / нарисовать как эта структура выглядит в памяти. Чему равен sizeof.

// в качестве примера запишем операции сложения и умножения для таких векторов

template<typename T, int N>
VectorN<T, N> operator +(const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    VectorN<T, N> rv;
    for (int i = 0; i < N; ++i)
        rv.data[i] = l.data[i] + r.data[i];
    return rv;    
}

template<typename T, int N>
T operator * (const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    T rv = 0;
    for (int i = 0; i < N; ++i)
        rv += l.data[i] * r.data[i];
    return rv;
}
```

Пример класса:

```c++
template<typename T, int N>
class RoundRobinQueue
{
private:
    std::array<T, N> arr;
    int begin_ix = 0;
    int end_ix = 0;

public:
    static int next_ix(const int ix) noexcept
    {
        return (ix + 1) % N;
    }
    
    void push(const T& item)
    {
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    T pop()
    {
        T item = arr[begin_ix];
        begin_ix = next_ix(begin_ix);
        return item;
    }

    bool empty() const noexcept
    {
        return begin_ix == end_ix;
    }
    
    bool full() const noexcept
    {
        return next_ix(end_ix) == begin_ix;
    }
};
```

Его использование:

```c++
RoundRobinQueue<int, 100> queue;
queue.push(1);
queue.push(2);
queue.push(3);
queue.pop();  // 1
queue.pop();  // 2
queue.pop();  // 3
```

__Вопрос__: Как обходить вопрос, что программист может позвать ещё один `queue.pop` ?

А ещё класс может иметь шаблонный метод, почему бы и нет.

```c++
#include <array>
#include <string>

template<typename T, int N>
class RoundRobinQueue
{
    ...

    template<typename U>
    void push(const U& item)
    {
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
};

std::string f()
{
    RoundRobinQueue<std::string, 100> queue;
    queue.push(std::string("run"))
    queue.push("Forest");
    queue.push("run");
    return queue.pop();
}
```

Закинуть пример на godbolt, показать, что генерируется 3 метода `push`, не забыть убрать оптимизации

Шаблонный конструктор - пожалуйста! Это ведь тоже метод

Шаблонный деструктор... пожалуй, нет, хватит.

__Вопрос__: какой подвох есть у такого шаблона?

```c++
template<typename T, typename U>
struct is_same
{
    static const bool value = false;
};

template<typename T>
struct is_same<T, T>
{
    static const bool value = true;
};

is_same<int, int>
is_same<int, float>
```

<br />

##### Частичная специализация шаблонов (для классов)

Например, можно прописать alias:

```c++
template<int N>
using RoundRobinStringsQueue = RoundRobinQueue<std::string, N>;


RoundRobinStringsQueue<100> queue;
```

А можно и изменить поведение класса

```c++
template<int N>
class RoundRobinQueue<std::string, N>
{
private:
    std::string arr[N];  // !!! here
    int begin_ix = 0;
    int end_ix = 0;

public:
    ...
    
    template<typename U>
    void push(const U& item)
    {
        std::cout << "push!";  // !!! here
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    ...
};
```

Такие сложности могут пригодиться для compile-time полиморфизма. С его классическим применением - type traits-ами - познакомимся позже.

<br />

##### just for fun: compile-time факториал

Теперь мы разбираемся в шаблонах достаточно чтобы посчитать факториал во время компиляции на шаблонах (разобрать пример, показать результат в godbolt).

Примечание: C++ значительно эволюционировал, и больше во время компиляции таким образом вычисления не проводят. Пример исключительно ученический. Compile-time вычисления будут, возможно, рассмотрены в курсе далее.

```c++
template<unsigned N>
struct Factorial
{
    static const int value = N * Factorial<N - 1>;
};

template<>
struct Factorial<0>
{
    static const int value = 1;
};

int main()
{
    return Factorial<10>::value;
}
```

__Вопросы__:
* Что делает следующий пример?

```c++
#include <cstdio>

template<unsigned N>
struct f
{
    static const int value = f<N-1>::value + f<N-2>::value;
};

template<>
struct f<0>
{
    static const int value = 0;
};

template<>
struct f<1>
{
    static const int value = 1;
};

int main()
{
    printf("%i\n", f<45>::value);
}
```

<br />

Во второй части курса про шаблоны:
* SFINAE
* variadic templates
* type traits
* tag dispatching (возможно)